In [25]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.svm import NuSVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix, roc_curve, f1_score
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.inspection import DecisionBoundaryDisplay

original = pd.read_csv("dados.csv")

In [26]:
le = preprocessing.LabelEncoder()

modificado = pd.DataFrame()
for x in original:
    if original[x].dtypes==object:
        modificado[x] = le.fit_transform(np.array(original[x]))
    else:
        modificado[x] = np.array(original[x])

clm = list(modificado.columns)
X = np.array(modificado[clm[0:-1]])
y = np.array(modificado[clm[-1]])
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [28]:
escala = preprocessing.MinMaxScaler(feature_range=(0, 1))
clfNN = MLPClassifier(hidden_layer_sizes=(3, 10,10,10), alpha=1e-5,verbose=True, learning_rate="adaptive",max_iter=500)
clasNN = make_pipeline(escala, clfNN)
clasNN.fit(X_train,y_train)
clasNN.score(X_test,y_test)
yp = clasNN.predict(X_test)

Iteration 1, loss = 0.34731306
Iteration 2, loss = 0.26261527
Iteration 3, loss = 0.25873050
Iteration 4, loss = 0.25611325
Iteration 5, loss = 0.25259873
Iteration 6, loss = 0.24992609
Iteration 7, loss = 0.24857737
Iteration 8, loss = 0.24747715
Iteration 9, loss = 0.24679689
Iteration 10, loss = 0.24628322
Iteration 11, loss = 0.24583270
Iteration 12, loss = 0.24536416
Iteration 13, loss = 0.24498747
Iteration 14, loss = 0.24478481
Iteration 15, loss = 0.24505096
Iteration 16, loss = 0.24433604
Iteration 17, loss = 0.24438875
Iteration 18, loss = 0.24450217
Iteration 19, loss = 0.24383770
Iteration 20, loss = 0.24372044
Iteration 21, loss = 0.24350176
Iteration 22, loss = 0.24340314
Iteration 23, loss = 0.24339981
Iteration 24, loss = 0.24315664
Iteration 25, loss = 0.24293998
Iteration 26, loss = 0.24318429
Iteration 27, loss = 0.24276112
Iteration 28, loss = 0.24271919
Iteration 29, loss = 0.24270080
Iteration 30, loss = 0.24241451
Iteration 31, loss = 0.24228889
Iteration 32, los

0.9673645320197044

In [31]:
sum(y_test)/len(y_test)

0.9367918902802623

In [52]:
from sklearn.utils import resample
y = modificado.COMPARECEU_AGENDAMENTO_EXAME
X = modificado.drop('COMPARECEU_AGENDAMENTO_EXAME', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X = pd.concat([X_train, y_train], axis=1)
NAO = X[X.COMPARECEU_AGENDAMENTO_EXAME==0]
SIM = X[X.COMPARECEU_AGENDAMENTO_EXAME==1]

NAO_upsampled = resample(NAO,
                          replace=True, # sample with replacement
                          n_samples=len(SIM), # match number in majority class
                          random_state=42)

upsampled = pd.concat([SIM, NAO_upsampled])
upsampled.COMPARECEU_AGENDAMENTO_EXAME.value_counts()

y_train_md = np.array(upsampled.COMPARECEU_AGENDAMENTO_EXAME)
X_train_md = np.array(upsampled.drop('COMPARECEU_AGENDAMENTO_EXAME', axis=1))


escala = preprocessing.MinMaxScaler(feature_range=(0, 1))
clfNN = MLPClassifier(hidden_layer_sizes=(3, 10,10,10), alpha=1e-5,verbose=True, learning_rate="adaptive",max_iter=500)
clasNN = make_pipeline(escala, clfNN)
clasNN.fit(X_train_md,y_train_md)
clasNN.score(np.array(X_test),np.array(y_test))
yp = clasNN.predict(np.array(X_test))

Iteration 1, loss = 0.69361216
Iteration 2, loss = 0.68981640
Iteration 3, loss = 0.68763039
Iteration 4, loss = 0.68442790
Iteration 5, loss = 0.67911541
Iteration 6, loss = 0.67219472
Iteration 7, loss = 0.66606407
Iteration 8, loss = 0.66075799
Iteration 9, loss = 0.65737333
Iteration 10, loss = 0.65543291
Iteration 11, loss = 0.65340245
Iteration 12, loss = 0.65154614
Iteration 13, loss = 0.65052900
Iteration 14, loss = 0.64936195
Iteration 15, loss = 0.64857568
Iteration 16, loss = 0.64822059
Iteration 17, loss = 0.64684685
Iteration 18, loss = 0.64635443
Iteration 19, loss = 0.64550916
Iteration 20, loss = 0.64559575
Iteration 21, loss = 0.64482442
Iteration 22, loss = 0.64410961
Iteration 23, loss = 0.64353473
Iteration 24, loss = 0.64324435
Iteration 25, loss = 0.64266316
Iteration 26, loss = 0.64221473
Iteration 27, loss = 0.64210302
Iteration 28, loss = 0.64189251
Iteration 29, loss = 0.64163386
Iteration 30, loss = 0.64183513
Iteration 31, loss = 0.64075508
Iteration 32, los

In [55]:
yp = clasNN.predict(np.array(X_test))
f1_score(yp,np.array(y_test))

0.7729323308270677